In [109]:
import time
import re
import math
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

- For newer versions of chromedriver use this
- https://support.google.com/chrome/thread/259906683/need-archives-of-chromedrivers-after-version-114-but-not-just-stable-or-beta-eg-121-0-6167-140?hl=en
- https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.57/win64/chromedriver-win64.zip
- just change your chrome version
- I think these are still beta so not all the functionality will be available

In [154]:
BASE_URL = "https://www.glassdoor.com/index.htm"
EMAIL = "YOUR EMAIL"
PASSWORD = "YOUR PASS"

In [66]:
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('http://www.google.com/');
#time.sleep(5) # Let the user actually see something!
#driver.quit()
get_html(body_inner_html,page_num=1)

In [148]:
def insert_before_pattern(main_string, pattern, insert_string):
    # Construct the regular expression pattern
    regex = re.compile(re.escape(pattern))

    # Find the index of the pattern
    match = regex.search(main_string)

    # If the pattern is found, insert the string before it
    if match:
        index = match.start()
        return main_string[:index] + insert_string + main_string[index:]
    else:
        # If pattern not found, simply return the original string
        return main_string

def replace_pattern_in_string(main_string, replacement):
    pattern = r'(_P\d+)'
    result = re.sub(pattern, replacement, main_string)
    return result

class Browser:
    def __init__(self):
        self.browser = webdriver.Chrome()
        self.next_url_suffix = "?filter.iso3Language=eng" #Change language if you want
        self.pattern = ".htm"

    def open_page(self, url: str):
        self.browser.get(url)

    def close_browser(self):
        self.browser.close()

    def add_input(self, by: By, value: str, text: str, send: bool = False):
        field = self.browser.find_element(by=by, value=value)
        field.send_keys(text)
        time.sleep(3)
        if send:
            field.send_keys(Keys.ENTER)

    def click_button(self, by: By, value: str):
        button = self.browser.find_element(by=by, value=value)
        button.click()
        time.sleep(5)

    def login_glassdoor(self, username: str, password: str):
        self.add_input(by=By.ID, value='inlineUserEmail', text=username)
        self.click_button(by=By.XPATH, value='//*[@id="InlineLoginModule"]/div/div/div[1]/div/div/div/div/form/div[2]/button')
        self.add_input(by=By.ID, value='inlineUserPassword', text=password)
        self.click_button(by=By.XPATH, value='//*[@id="InlineLoginModule"]/div/div/div[1]/div/div/div/div/form/div[4]/button')

    def search_company(self,company_name: str):
        self.click_button(by=By.XPATH, value = '//*[@id="UtilityNav"]/div[1]/button')
        self.add_input(by=By.ID, value='sc.keyword', text=company_name, send = True)
        self.click_button(by=By.XPATH, value = '//*[@id="Discover"]/div/div/div[1]/div[1]/div/a')
        self.click_button(by=By.XPATH, value = '//*[@id="EmpLinksWrapper"]/div[2]/div/div[1]/a[1]')
        ## Clear All Filters, this is inconsistent for some reason
        # self.click_button(by=By.XPATH, value = '/html/body/div[3]/div/div[1]/div[2]/main/div[1]/div[1]/div/div[1]/div[4]/div[1]/a')

    def get_html(self, page_num:int):
        """We want to write the HTML rather than do all of the parsing here"""
        body_inner_html = self.browser.find_element(By.TAG_NAME, 'body')
        html = body_inner_html.get_attribute('innerHTML')
        with open(f"html/page_{page_num}.html","w", encoding="utf-8") as out:
            out.write(html)

    def construct_urls(self,starting_url:str,num_urls:int):
        curr_url = starting_url
        for i in range(2,num_urls + 1):
            result = replace_pattern_in_string(curr_url, "")
            if self.next_url_suffix not in result:
                result+=self.next_url_suffix
            insert_string = f"_P{i}"
            result = insert_before_pattern(result, pattern = self.pattern, insert_string = insert_string)
            yield result
            curr_url = result

    def collect_reviews(self,num_reviews:int):
        # Start by getting the current reviews
        self.get_html(page_num=1)
        # Now construct the next URL
        # You will iterate through the subsequent pages, no need to click 
        # Figure out how many urls to construct based on num_reviews and num_displayed
        # we know it is ~1,700 for LLNL
        num_urls = math.ceil(num_reviews//10)
        for url in self.construct_urls(starting_url = self.browser.current_url, num_urls= num_urls):
            print(url)
            self.open_page(url = url)
            time.sleep(3)
            #extract the page number
            match = re.search(r'_P(\d+)', url)
            if match:
                # Extract the number from the match
                number = match.group(1)
                self.get_html(page_num=number)
            else:
                raise Exception(f"Something wrong with the URL!!! {url}")

In [149]:
browser = Browser()

In [150]:
browser.open_page(url = BASE_URL)

In [151]:
browser.login_glassdoor(username = EMAIL, password = PASSWORD)

In [152]:
browser.search_company(company_name = "Lawrence Livermore National Laboratory")

In [153]:
browser.collect_reviews(num_reviews = 30)

https://www.glassdoor.com/Reviews/Lawrence-Livermore-National-Laboratory-Reviews-E35235_P2.htm?filter.iso3Language=eng
https://www.glassdoor.com/Reviews/Lawrence-Livermore-National-Laboratory-Reviews-E35235_P3.htm?filter.iso3Language=eng
